# Finding similar company name and auto matching them

This program will use NLP and ML technique to match similar company names. Matching form common words like "LTD" and "COMPANY" will be discounted autometically in the algorithm.

Library used:
* pandas
* fuzzywuzzy

## Load the table in pandas DataFrame

The data we used is found on http://download.companieshouse.gov.uk/en_output.html it is an openly licensed publicly avalible dataset that contains a list of registered (limited liability) companies in Great Britain. *(the version shown here is snapshot of May 2018)*

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
df = pd.read_csv("BasicCompanyDataAsOneFile-2018-05-01.csv")
df.head()

C:\Users\921345\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,Private Limited Company,Active,United Kingdom,NaN,11/09/2012,30.0,9.0,30/06/2018,30/09/2016,DORMANT,09/10/2016,11/09/2015,0,0,0,0,99999 - Dormant Company,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/08209948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2019,11/09/2017
1,!NNOV8 LIMITED,11006939,NaN,NaN,C/O FRANK HIRTH 1ST FLOOR,236 GRAY'S INN ROAD,LONDON,NaN,UNITED KINGDOM,WC1X 8HB,Private Limited Company,Active,United Kingdom,NaN,11/10/2017,31.0,3.0,11/07/2019,NaN,NO ACCOUNTS FILED,08/11/2018,NaN,0,0,0,0,62090 - Other information technology service a...,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/11006939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24/10/2019,NaN
2,!NSPIRED LTD,SC421617,NaN,NaN,26 POLMUIR ROAD,NaN,ABERDEEN,NaN,UNITED KINGDOM,AB11 7SY,Private Limited Company,Active,United Kingdom,NaN,11/04/2012,30.0,3.0,30/12/2018,30/03/2017,TOTAL EXEMPTION FULL,09/05/2017,11/04/2016,0,0,0,0,70229 - Management consultancy activities othe...,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/SC421617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/04/2020,11/04/2018
3,!NVERTD DESIGNS LIMITED,09152972,NaN,NaN,55A HIGH STREET,NaN,SILSOE,BEDFORDSHIRE,NaN,MK45 4EW,Private Limited Company,Active,United Kingdom,NaN,30/07/2014,31.0,7.0,30/04/2019,31/07/2017,NaN,27/08/2016,30/07/2015,0,0,0,0,58190 - Other publishing activities,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/09152972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13/08/2020,30/07/2017
4,!OBAC LIMITED,FC031362,NaN,NaN,1ST AND 2ND FLOORS ELIZABETH HOUSE,LES RUETIES BRAYES,ST PETER PORT,GUERNSEY,GUERNSEY,GY1 1EW,Other company type,Active,CHANNEL ISLANDS,NaN,30/11/2012,31.0,12.0,NaN,31/12/2016,GROUP,NaN,NaN,0,0,0,0,None Supplied,NaN,NaN,NaN,0,0,http://business.data.gov.uk/id/company/FC031362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*This is a huge table with lots of rows, it may take a while to load*

In [2]:
df.columns

Index(['CompanyName', ' CompanyNumber', 'RegAddress.CareOf',
       'RegAddress.POBox', 'RegAddress.AddressLine1',
       ' RegAddress.AddressLine2', 'RegAddress.PostTown', 'RegAddress.County',
       'RegAddress.Country', 'RegAddress.PostCode', 'CompanyCategory',
       'CompanyStatus', 'CountryOfOrigin', 'DissolutionDate',
       'IncorporationDate', 'Accounts.AccountRefDay',
       'Accounts.AccountRefMonth', 'Accounts.NextDueDate',
       'Accounts.LastMadeUpDate', 'Accounts.AccountCategory',
       'Returns.NextDueDate', 'Returns.LastMadeUpDate',
       'Mortgages.NumMortCharges', 'Mortgages.NumMortOutstanding',
       'Mortgages.NumMortPartSatisfied', 'Mortgages.NumMortSatisfied',
       'SICCode.SicText_1', 'SICCode.SicText_2', 'SICCode.SicText_3',
       'SICCode.SicText_4', 'LimitedPartnerships.NumGenPartners',
       'LimitedPartnerships.NumLimPartners', 'URI', 'PreviousName_1.CONDATE',
       ' PreviousName_1.CompanyName', ' PreviousName_2.CONDATE',
       ' PreviousName_2.C

In [3]:
df['RegAddress.PostTown'].value_counts().head(30)

LONDON                 767269
MANCHESTER              71854
BIRMINGHAM              70114
GLASGOW                 51945
BRISTOL                 47288
EDINBURGH               45811
LEEDS                   40947
LIVERPOOL               33040
NOTTINGHAM              32368
LEICESTER               31233
SHEFFIELD               26803
WARRINGTON              24281
BRIGHTON                23064
HARROW                  22026
CARDIFF                 21907
COVENTRY                21771
READING                 21461
MILTON KEYNES           20348
SOUTHAMPTON             19225
ILFORD                  18380
NORWICH                 17227
STOCKPORT               17116
NORTHAMPTON             16263
CROYDON                 16013
BOLTON                  15950
CAMBRIDGE               15889
BELFAST                 15627
NEWCASTLE UPON TYNE     15569
DERBY                   15241
POOLE                   14982
Name: RegAddress.PostTown, dtype: int64

## Frequency of words
Since we have lots of companies, we will only use companies in Cambridge as an example.

First we find the 30 most common words in all company names. As we will be expecting them to be repeating a lot even in companies that is not the same, we cannot match company names using them. The way we do it is we will deduct the matching score of a pair if any keywords is present in the names.

In [4]:
from collections import Counter
all_names = df['CompanyName'][df['RegAddress.PostTown']=='CAMBRIDGE'].unique()
names_freq = Counter()
for name in all_names:
    names_freq.update(str(name).split(" "))
key_words = [word for (word,_) in names_freq.most_common(30)]
print(key_words)

['LIMITED', 'LTD', 'CAMBRIDGE', 'SERVICES', 'MANAGEMENT', '&', 'COMPANY', 'THE', 'CONSULTING', 'LTD.', 'SOLUTIONS', 'AND', 'PROPERTY', 'UK', 'LLP', '(CAMBRIDGE)', 'CONSULTANCY', 'GROUP', 'HOLDINGS', 'CONSULTANTS', 'ASSOCIATES', 'COMPOSITES', 'ENGINEERING', 'DEVELOPMENTS', 'INTERNATIONAL', 'OF', 'DESIGN', 'TECHNOLOGY', 'PROPERTIES', '(UK)']


In [5]:
len(all_names)

15889

## Matching by Grouping
Then we group the names by their 1st character. As the list is too long, it will take forever to match them all at once (15889 x 15889 pairs to consider). The work around is to match them by groups, assuming if the names are not matched at the 1st character, it is unlikely that they are the same name. 

In [6]:
all_main_name = pd.DataFrame(columns=['sort_gp','names','alias','score'])
all_names.sort()
all_main_name['names'] = all_names
all_main_name['sort_gp'] = all_main_name['names'].apply(lambda x: x[0])

## Fuzzy Matching
Here for each group, we use `fuzzywuzzy.token_sort_ratio` to matching the names. Different form the basic `fuzzywuzzy.ratio` which use Levenshtein Distance to calculate the differences, it allow the token (words) in a name to swap order and still give a 'perfect' match. (ref: https://github.com/seatgeek/fuzzywuzzy)

In [7]:
from fuzzywuzzy import fuzz

all_sort_gp = all_main_name['sort_gp'].unique()

def no_key_word(name):
    """check if the name contain the keywords in travel company"""
    output = True
    for key in key_words:
        if key in name:
            output = False
    return output

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(['sort_gp']).get_group(sortgp)
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i]
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                fuzz_socre = fuzz.token_sort_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                if not no_key_word(all_main_name['names'].iloc[j]):
                    fuzz_socre -= 10
                if (fuzz_socre > 85):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_socre
                    
        if i % (len(all_names)//10) == 0:
            print("progress: %.2f" % (100*i/len(all_names)) + "%")
                
all_main_name.to_csv('company_in_cambridge.csv')

progress: 0.00%
progress: 9.99%
progress: 19.99%
progress: 29.98%
progress: 39.98%
progress: 49.97%
progress: 59.97%
progress: 69.96%
progress: 79.95%
progress: 89.95%
progress: 99.94%


In [8]:
all_main_name[(all_main_name['names']!=all_main_name['alias']) & (all_main_name['alias'].notna())]

,sort_gp,names,alias,score
761,A,AMADEUS EII LP,AMADEUS EI LP,96
762,A,AMADEUS EIII LP,AMADEUS EI LP,93
763,A,AMADEUS HI LP,AMADEUS EI LP,92
766,A,AMADEUS II 'A',AMADEUS I,86
767,A,AMADEUS II 'B',AMADEUS I,86
768,A,AMADEUS II 'C',AMADEUS I,86
769,A,AMADEUS III,AMADEUS I,90
773,A,AMADEUS IV EARLY STAGE FUND B LP,AMADEUS IV EARLY STAGE FUND A LP,94
776,A,AMADEUS JI LP,AMADEUS EI LP,92
777,A,AMADEUS LI LP,AMADEUS EI LP,92


The result is saved in a csv file locally for future inspection and further experimentation. Inspecting the result, the matches consisted of 3 groups:

1. they are usually differ in spelling by 1 character: missing an 'L' or 'I' or 'S'
2. highly similar names: 'No.3' instead of 'No.2' or 'EB' instread of 'EH'
3. fairly similar names: 'HAMMER AND THONGS PRODUCTIONS LIMITED' and 'HAMMER AND TONG PRODUCTIONS LIMITED'

For type 1 and 2 matches it could be the same company, the diffeernce in names could be an intentional alteration or simply a typo. But it is not likely the same company for type 3 matched, it seems more like a coincidnce. 

To further confirm, manual work need to be done but this program saves a lot of manual work hours.

In [9]:
all_main_name[(all_main_name['names']!=all_main_name['alias']) & (all_main_name['alias'].notna())].shape[0]

57

In [10]:
len(all_main_name['alias'].unique())

15832

By applying the fuzzy matching, 57 names are caught similar to another name, which is less then 1% of the total. By using this program names that need checking drastically reduce form 15889 total to only 57.